# Code get memory usage information from GPU using gpustat command

## sample csv file is also attached

In [ ]:
import time
import datetime
import subprocess
import pandas as pd
import json

# Create a DataFrame to store memory consumption data
memory_df = pd.DataFrame(columns=["Time"])

# Function to get GPU memory usage
def get_gpu_memory_usage():
    cmd = "gpustat --json"
    result = subprocess.check_output(cmd.split()).decode("utf-8")
    gpu_info = json.loads(result)["gpus"]
    gpu_memory_usage = []
    for gpu in gpu_info:
        for process in gpu["processes"]:
            gpu_memory_usage.append({"GPU": gpu["index"], "PID": process["pid"], "User": process["username"], "Memory": process["gpu_memory_usage"]})
    return gpu_memory_usage

# Track memory consumption at different points in your code
# Example:
for i in range(10):
    # Your code here

    # Get GPU memory usage and add it to the DataFrame
    gpu_memory_usage = get_gpu_memory_usage()
    current_time = time.time()
    human_readable_time = datetime.datetime.fromtimestamp(current_time).strftime('%Y-%m-%d %H:%M:%S')
    memory_data = {"Time": human_readable_time}  # Convert timestamp to human-readable time
    for gpu_usage in gpu_memory_usage:
        gpu = gpu_usage["GPU"]
        pid = gpu_usage["PID"]
        username = gpu_usage["User"]
        memory = gpu_usage["Memory"]
        column_name = f"GPU {gpu} (PID {pid}) - User: {username}"
        memory_data[column_name] = memory

    # Simulate new process starting during code execution
    if i == 5:
        new_process = {"GPU": 1, "PID": 1234, "User": "new_user", "Memory": 512}
        column_name = f"GPU {new_process['GPU']} (PID {new_process['PID']}) - User: {new_process['User']}"
        memory_data[column_name] = new_process['Memory']

    memory_df = pd.concat([memory_df, pd.DataFrame(memory_data, index=[0])], ignore_index=True)

# Save the DataFrame to a CSV file
memory_df.to_csv("memory_consumption.csv", index=False)